## Script that gets Medicine Label Images using Web Scraping in Python

In [2]:
import sys
sys.version

'3.7.7 (default, May  7 2020, 21:25:33) \n[GCC 7.3.0]'

In [4]:
! pip install bs4

     |████████████████████████████████| 115 kB 879 kB/s eta 0:00:01
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1272 sha256=8e360e76c1066d10499017a4414f2a8262709d3c50060b8663b85502b070e318
  Stored in directory: /home/vari/.cache/pip/wheels/0a/9e/ba/20e5bbc1afef3a491f0b3bb74d508f99403aabe76eda2167ca
Successfully built bs4


In [6]:
! pip install selenium
# saved the chromedriver to ~/Documents/AWS

In [13]:
! pip install Pillow

     |████████████████████████████████| 2.2 MB 752 kB/s eta 0:00:01     |█████████████▊                  | 942 kB 496 kB/s eta 0:00:03


In [33]:
import requests
from bs4 import BeautifulSoup as bs
import selenium
import os, time, io, hashlib
from PIL import Image

In [17]:
from selenium import webdriver
wd = webdriver.Chrome('./chromedriver')

In [9]:
driver.get('https://google.com')

In [12]:
search_box = driver.find_element_by_css_selector('input.gLFyf')
search_box.send_keys('medicine label')

---

## Function to get img urls

In [14]:
def fetch_image_urls(query:str, max_links_to_fetch:int, wd:webdriver, sleep_between_interactions:int=1):
    def scroll_to_end(wd):
        wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(sleep_between_interactions)    
    
    # build the google query
    search_url = "https://www.google.com/search?safe=off&site=&tbm=isch&source=hp&q={q}&oq={q}&gs_l=img"

    # load the page
    wd.get(search_url.format(q=query))

    image_urls = set()
    image_count = 0
    results_start = 0
    while image_count < max_links_to_fetch:
        scroll_to_end(wd)

        # get all image thumbnail results
        thumbnail_results = wd.find_elements_by_css_selector("img.Q4LuWd")
        number_results = len(thumbnail_results)
        
        print(f"Found: {number_results} search results. Extracting links from {results_start}:{number_results}")
        
        for img in thumbnail_results[results_start:number_results]:
            # try to click every thumbnail such that we can get the real image behind it
            try:
                img.click()
                time.sleep(sleep_between_interactions)
            except Exception:
                continue

            # extract image urls    
            actual_images = wd.find_elements_by_css_selector('img.n3VNCb')
            for actual_image in actual_images:
                if actual_image.get_attribute('src') and 'http' in actual_image.get_attribute('src'):
                    image_urls.add(actual_image.get_attribute('src'))

            image_count = len(image_urls)

            if len(image_urls) >= max_links_to_fetch:
                print(f"Found: {len(image_urls)} image links, done!")
                break
        else:
            print("Found:", len(image_urls), "image links, looking for more ...")
            time.sleep(30)
            return
            load_more_button = wd.find_element_by_css_selector(".mye4qd")
            if load_more_button:
                wd.execute_script("document.querySelector('.mye4qd').click();")

        # move the result startpoint further down
        results_start = len(thumbnail_results)

    return image_urls

## Function to download image

In [15]:
def persist_image(folder_path:str,url:str):
    try:
        image_content = requests.get(url).content

    except Exception as e:
        print(f"ERROR - Could not download {url} - {e}")

    try:
        image_file = io.BytesIO(image_content)
        image = Image.open(image_file).convert('RGB')
        file_path = os.path.join(folder_path,hashlib.sha1(image_content).hexdigest()[:10] + '.jpg')
        with open(file_path, 'wb') as f:
            image.save(f, "JPEG", quality=85)
        print(f"SUCCESS - saved {url} - as {file_path}")
    except Exception as e:
        print(f"ERROR - Could not save {url} - {e}")

## Function to get the URL and download it

In [22]:
def getdata(search_term:str,driver_path:str,target_path='./images',number_images=5):
    target_folder = os.path.join(target_path,'_'.join(search_term.lower().split(' ')))

    if not os.path.exists(target_folder):
        os.makedirs(target_folder)

    with webdriver.Chrome('./chromedriver') as wd:
        res = fetch_image_urls(search_term, number_images, wd=wd, sleep_between_interactions=0.5)
        
    for elem in res:
        persist_image(target_folder,elem)

In [34]:
searchterm = 'medicine label photos'
pathtoimg = './scrapedata'
imgcount = 2
getdata(search_term = searchterm,
        driver_path = pathtoimg,
        number_images = imgcount)

Found: 100 search results. Extracting links from 0:100
Found: 2 image links, done!
SUCCESS - saved https://www.fda.gov/media/75906/download - as ./images/medicine_label_photos/baf949d3ec.jpg
SUCCESS - saved https://i.ytimg.com/vi/XL-AjG0FUCg/maxresdefault.jpg - as ./images/medicine_label_photos/2b9808f654.jpg


In [36]:
medicine_names = ['pyridoxine hydrochloride',
                  'metylcobalamin pyridoxine',
                  'pyridoxine hydrochloride',
                  'crocin',
                  'vicks action',
                  'paracetamol',
                  'cetzine'
]

Website with good photos: buy-pharma.md